# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [1]:
!pip install transformers
!apt install swig
# Sentencepieceのインストール
!pip install sentencepiece
!pip install -U easynmt
!pip install -U protobuf~=3.20.0
#!pip install -U protobuf

from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


# PyTorchとGPU設定

In [2]:
!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import re
import csv
import glob
import warnings
import numpy as np
warnings.filterwarnings('ignore')
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [4]:
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
target_genre = ["dokujo-tsushin", "it-life-hack"] 
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
#target_genre = ["dokujo-tsushin", "peachy"] 
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [5]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1740, 4)


,media_name,label,title,sentence
1257,it-life-hack,1,地方の元気が日本の元気に！ニコニコ町会議佐賀県唐津市呼子町に1万6千人が来,ドワンゴおよびニワンゴによると、8月5日日にニコニコ町会議も参加して開催された佐賀県唐津市呼...
194,dokujo-tsushin,0,友達が多い人って、実は性格がよくない！,子供の頃、友達はたくさん作りなさいと学校でも家庭でも教わらなかっただろうか？偏らず、いろんな...
354,dokujo-tsushin,0,なぜ入れる!?否定派、肯定派割れるタトゥ,蒸し暑い日が続いているが、梅雨明けまでもう少し。夏はすぐそこまで来ている。休暇やイベントが多...
1382,it-life-hack,1,今度の金環食を見逃さない！来る5月21日の金環食を楽しむアプリiPhoneでチャンスを掴,5月21日に日本では九州から関東にかけて太平洋側で金環食が観測できます。太平洋側のどこで、ど...
1366,it-life-hack,1,歌＋歌歌見事な才能に感嘆！え？二曲を同時再生するとまったく別の曲が,ハロープロジェクト所属のグループBerryz工房が歌うBecausehappinessと℃-...
179,dokujo-tsushin,0,オトナ女子コミック部元AV女優がぶっちゃけるエロカワ漫画アラサーちゃ,女子高育ち恋のから騒ぎ出演AVデビューと数の女の園をくぐり抜け、現在は元AV女優ライターとい...
1718,it-life-hack,1,超コンパクトで充電ケーブルいらず！わずか92gのモバイルバッテリーイケショップのレア,スマホ対応のモバイルバッテリーは、技術の進歩もドッグイヤーで進み、バッテリーの大容量化により...
1724,it-life-hack,1,スマホやタブレットの活用グッズが満載！バッテリー切れから車載対応までイケショップで解決,ゴールデンウィークに突入しましたが、みなさんいかがお過ごしでしょうか？これから旅行に出る方も...
1131,it-life-hack,1,アナタなら10個のCPU何に使う？SandyBridge-EとX79マザーの使い道を大募集,ソーシャルレビューコミュニティzigsowジグソーは、SandyBridge-EでLGA20...
1020,it-life-hack,1,愛される英王子裸で敬礼！ハリー英国王子に対する英国軍人の大いなる,このサイトSalute4Harryという。SaluteforHarry.でforが付いてるの...


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [6]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [7]:
type(sentences)

numpy.ndarray

# Data Augmentation

同義語で置換, randomな同義語挿入, randomなワード移動, randomなワード削除, 逆翻訳


    bert no mask ni tango ireru yatsu mo ikesou
    
    https://qiita.com/tchih11/items/aef9505d26d1bf06a04c
    https://github.com/tchih11/qiita_eda
    
    random de iroiro yaru yatsu, kittekara yarunoka kirumaeni yarunoka wakarann
    
    ---
    
    atoha honnyaku desune
    
    https://tech.excite.co.jp/entry/2022/08/19/191151

In [8]:
from easynmt import EasyNMT
model_trans = EasyNMT('m2m_100_1.2B')

In [9]:
sentence = 'Hello, it\'s good weather today!'

print(model_trans.translate(sentence, target_lang='ja', max_new_tokens=1000))

こんにちは、今日はいい天気です!


In [10]:
btrans = sentences
atrans = np.zeros(len(sentences), dtype=object)

for sentran in sentences:
    atrans=np.append(atrans, model_trans.translate(sentran, target_lang='en', max_new_tokens=10000))
    if len(atrans) % 10 == 0:
        print('hoge')

hoge
hoge
hoge
hoge
hoge
hoge
hoge
hoge
hoge


/opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/Indexing.cu:650: indexSelectSmallIndex: block: [3,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/Indexing.cu:650: indexSelectSmallIndex: block: [3,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/Indexing.cu:650: indexSelectSmallIndex: block: [3,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/Indexing.cu:650: indexSelectSmallIndex: block: [3,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cuda/Indexing.cu:650: indexSelectSmallIndex: block: [3,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pyt

RuntimeError: CUDA error: device-side assert triggered

colabo dato ugoku!